First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import cv2
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
 

#CATEGORIES = ['apex']



In [2]:
import torch.nn as nn
import torch.nn.functional as F
 
class Net(nn.Module):
 
# 使用するオブジェクトを定義
    def __init__(self):
        super(Net, self).__init__()
 
# 畳み込み層
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
 
# プーリング層
        self.pool = nn.MaxPool2d(kernel_size=2)
 
# 全結合層
        self.fc1 = nn.Linear(256*2*2, 128)
        self.fc2 = nn.Linear(128, 8)

# 順伝播
    def forward(self, x):
        h = self.pool(F.relu(self.conv1(x)))  # (3*64*64) -> (64*28*28) -> (32*14*14)
        h = self.pool(F.relu(self.conv2(h)))  # (64*14*14) -> (128*12*12) -> (128*6*6)
        h = self.pool(F.relu(self.conv3(h)))  # (128*6*6) -> (256*4*4) -> (256*2*2)
        h = torch.flatten(h, 1)  # (256*2*2) -> 1,600次元のベクトル
        h = F.relu(self.fc1(h))  # 1600 -> 128
        y = self.fc2(h)  # 128 -> 10
        return y
 
# ネットワークのインスタンスを作成
net = Net()
 
# 損失関数
criterion = nn.CrossEntropyLoss()
 
# 最適化手法
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)



Next, load the saved model.  Enter the model path you used to save.

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
model = net.to(device)
 
#model = model.cuda().eval().half()

In [4]:
model.load_state_dict(torch.load('mobel_new.pth'))#mobel_new.pthが最新

<All keys matched successfully>

#Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme #for more details.

> This optimization process can take a couple minutes to complete. 

In [5]:
#from torch2trt import torch2trt

#data = torch.zeros((1, 3, 224, 224)).cuda().half()

#model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [6]:
#torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

Load the optimized model by executing the cell below

In [7]:
#import torch
#from torch2trt import TRTModule

#model_trt = TRTModule()
#model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))#モデル読み込み

Create the racecar class

In [8]:
from nvidia_racecar import NvidiaRacecar
#from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

Create the camera class.

In [9]:
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [37]:
#終了
car.throttle = 0.0
car.steering = 0.0

In [36]:
from utils import preprocess
import numpy as np
import torchvision.transforms as transforms
import time
import PIL.Image
import time
STEERING_MAX = 3.0 #ステアリングの最大値
STEERING_GAIN = 0.9  #どの程度曲がるか
STEERING_BIAS = -0.0 #ステアリングの中心
THROTTLE_GAIN = 0.5 #スロットルの強さ
#THROTTLE_BIAS = -0.2
label=0
pre_label=-1
time_2=0
#car.throttle = -0.45  #前進する強さ(初期値)
while True:
    for i in range(10):
        camera.read()
    image = camera.read()#opencv bgr
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image) #pil rgb
    #image = preprocess(image).half()#浮動小数点に変換
    transform = transforms.Compose([transforms.Resize(32),transforms.ToTensor()])
    image = transform(image)
    image=torch.unsqueeze(image,0)
    #print(image.shape)
    #image=image.half()
    image=image.to(device)
    label = model(image).to(device).detach().cpu().numpy().flatten()
    label=np.argmax(label)
    if label != pre_label:
        print(label)
        pre_label=label
    if label == 0:
      car.steering = 0.0
      #car.throttle = 0.3
      car.throttle = THROTTLE_GAIN * 0.7
      time.sleep(0.02/THROTTLE_GAIN)
    elif label == 1:
      car.steering = STEERING_GAIN * STEERING_MAX/2.0
      #car.throttle = 0.35
      car.throttle = THROTTLE_GAIN * 0.7
      time.sleep(0.01/THROTTLE_GAIN)
    elif label == 2:
      car.steering = -STEERING_GAIN * STEERING_MAX/2.0
      #car.throttle = 0.35
      car.throttle = THROTTLE_GAIN * 0.7
      time.sleep(0.01/THROTTLE_GAIN)
    elif label == 3:
      car.steering = 0.0
      #car.throttle = -0.2
      car.throttle = THROTTLE_GAIN * -0.7
    elif label == 4:
      car.steering = STEERING_GAIN * STEERING_MAX/2.0
      #car.throttle = -0.2
      car.throttle = THROTTLE_GAIN * -0.7
    elif label == 5:
      car.steering = -STEERING_GAIN * STEERING_MAX/2.0  #成功時 0.8
      #car.throttle = -0.2
      car.throttle = THROTTLE_GAIN * -0.7
    elif label == 6:
      car.steering = STEERING_MAX/2.0
      #car.throttle = -0.2
      car.throttle = THROTTLE_GAIN * -0.7
    elif label == 7:
      car.steering = -STEERING_MAX/2.0 #成功時　1.0
      #car.throttle = -0.2
      car.throttle = THROTTLE_GAIN * -0.7

3
4
5
7
5
3
5
7
5
3
5
3
5
7
5
3
4
6
4
3
7
0
7
0
7
0
7
1
7
1
7
0
7
5
3
4
3
7
0
7
1
7
3
5
3
4
3
5
7
5
3
7
5
3
7
5
3
6
4
3
7
0
7
0
1
7
0
7
0
7
1
7
0
7
5
3
4
3
7
1
0
7
0
7
5
3
4
6
4
3
7
0
7
5
3
5
7
5
3
7
5
3
6
4
3
5
7
1
7
1
7
5
3
4
7
1
7
5
3
6
4
3
5
3
7
5
3
7
5
3
7
5
3
4
3
6
2
4
3
7
0
7
1
7
0
7
0
7
1
7
0
7
5
3
4
7
0
7
5
3
4
6
4
3
5
7
5
3
7
5
3
5
7
3
5
3
4
3
4
6
4
3
5
7
0
7
0
7
1
7
5
3
7
5
3
4
3
5
7
5
3
5
7
5
3
5
3
6
0
6
4
3
5
7
0
7
1
7
5
3
5
7
5
3
5
3
4
5
7
5
3
5
3
5
3
5
3
6
4
3
5
3
7
0
7
1
7
5
3
4
3
7
0
7
5
3
6
4
3
5
7
5
3
7
5
3
5
7
5
3
4
6
4
3
7
0
7
0
7
0
7
0
7
5
3
4
7
5
3
4
3
7
5
3
5
7
5
3
5
3
7
5
3
4
6
4
3
7
0
7
1
7
5
3
4
7
0
7
0
7
5
3
4
3
7
5
3
7
5
3
7
5
3
4
6
4
3
7
0
7
0
7
0
7
1
7
1
7
5
3
5
3
7
5
3
5
3
5
7
5
3
5
7
5
3
5
3
6
0
6
0
2
6
4
3
7
0
7
0
7
1
7
5
3
7
5
3
4
6
4
3
7
0
7
5
3
5
3
5
7
5
3
5
3
6
0
2
4
3
5
3
5
3
5
7
5
3
4
3
4
3
4
6
4
3
4
3
4
3
6
2
6
4
3
4
6
2
6
2
4
3
7
5
7
5
3
4
3
4
3
4
6
4
3
6
4
3
5
3
4
6
2
0
6
4
3
6
0
2
6
4
5
7
5
3
4
6
4
3
4
6
4
3
4
3
4
3
6
2
4
3
4
3
6
2
6
2
4
3
7


KeyboardInterrupt: 

In [23]:
#終了
car.throttle = 0.0
car.steering = 0.0

In [ ]:
print(device)